## Scraping "Data Science" Job Descriptions from indeed.com

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd

In [2]:
# Skills & Place of Work
skill = 'Data+Science'
place = 'Pakistan'
no_of_pages = 20

indeed_posts = []


for page in range(no_of_pages):
    # Connecting to Indeed
    url = 'https://pk.indeed.com/jobs?q=' + skill + '&l=' + place + '&start=' + str(page * 10)
    try:
        driver = webdriver.Chrome()
        # Create a new Chrome driver instance
        driver.get(url)

        # Wait for the page to load (you might need to adjust the waiting time)
        driver.implicitly_wait(10)

        # Get the page source after it's loaded
        html = driver.page_source

        # Close the driver after getting the page source
        # driver.quit()
    except Exception as e:
        print(e)

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'lxml')

    # Outer Most Entry Point of HTML:
    outer_most_point = soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

    # "UL" lists where the data are stored:
    for i in outer_most_point.find_all('li', class_='css-5lfssm eu4oa1w0'):
        # Job Title:
        job_title = i.find('h2', class_='jobTitle css-14z7akl eu4oa1w0')
        
        if job_title is not None:
            jobs = job_title.find('a').text

        # Company Name:
        if i.find('span',{'class':'css-92r8pb eu4oa1w0'}) != None:
            company = i.find('span',{'class':'css-92r8pb eu4oa1w0'}).text   

        # Links: these Href links will take us to full job description
        if i.find('a') is not None:
            links = i.find('a', class_='jcs-JobTitle')['href']


        # Put everything together in a list of lists for the default dictionary
        indeed_posts.append([jobs, company, links])
driver.quit()
indeed_headers = ['Job Title', 'Company', 'Link']
# create a pandas dictionary
indeed_df = pd.DataFrame(indeed_posts, columns=indeed_headers)

In [3]:
indeed_df

,Job Title,Company,Link
0,Microsoft Excel Expert,Progressive Resource,/rc/clk?jk=ca30fa0935fdfc32&bb=Yn3GPrx_lF8e-Dh...
1,Admin Officer,The University of Lahore,/rc/clk?jk=14ee818fb1c66447&bb=Yn3GPrx_lF8e-Dh...
2,Science LAB Assistant,LACAS SCHOOL NETWORK,/rc/clk?jk=075cb97c525f8753&bb=Yn3GPrx_lF8e-Dh...
3,Creative Content Writer,Adequate Solutions,/rc/clk?jk=4fe01630a3dbbaca&bb=Yn3GPrx_lF8e-Dh...
4,Associate Data Scientist,i2c inc.,/rc/clk?jk=d023edfdbd0f3669&bb=Yn3GPrx_lF8e-Dh...
...,...,...,...
355,Sr. Data Engineer (Azure Focus),RemcoSol Technologies,/rc/clk?jk=414706150bb18679&bb=VXjSJBZPXwnUpnN...
356,Nodejs Backend Developer,See Sight Tours,/rc/clk?jk=f23d34bd0d5d26e4&bb=VXjSJBZPXwnUpnN...
357,SOC Analyst - L2,Zones,/rc/clk?jk=7faf0b0814995252&bb=VXjSJBZPXwnUpnN...
358,DevOps Systems Administrator,ThinkDone Solutions,/rc/clk?jk=d7a31e6349fd0782&bb=VXjSJBZPXwnUpnN...


In [4]:
indeed_df['Link'].nunique()
indeed_df = indeed_df.drop_duplicates(subset='Link')
indeed_df.to_csv('indeed_links.csv', index=False)

In [5]:
job_descr_txt=[]

# convert Series to lists of strings
my_super_fun_Indeed_links = list(indeed_df['Link'])

# iterator will be our index value for default_dict_list
for i in range(len(my_super_fun_Indeed_links)):
    driver = webdriver.Chrome()
    
    url_href='https://pk.indeed.com' + my_super_fun_Indeed_links[i]
    driver.get(url_href)
    driver.implicitly_wait(10)
    html_ = driver.page_source

    soup_ = BeautifulSoup(html_, "html.parser")

    
    for ii in soup_.find('div',{'class':'jobsearch-jobDescriptionText'}):
        try:
            job_descr_txt.append([i,''.join(ii.text.strip())])
        except AttributeError:
            job_descr_txt.append([i,''])
driver.quit()
job_descr_txt

[[0, 'Key Responsibilities:'],
 [0, ''],
 [0,
  'Collect data from various sources, ensuring accuracy and completeness for analysis.\nAnalyze information to uncover insights, trends, and valuable findings.\nCreate well-structured and visually engaging reports and summaries to communicate research outcomes.\nSupport research efforts by gathering relevant data and assisting in investigations.\nCollaborate with teams to provide insights and solutions based on research.\nStay updated on research methods and tools to enhance your expertise.\nEffectively present findings and recommendations for different audiences.'],
 [0, ''],
 [0, 'Qualifications:'],
 [0, ''],
 [0,
  "Education & Skills: Bachelor's degree in a relevant field, such as Business, Social Sciences, or a related discipline and Strong ability to analyze data and purify it into meaningful insights."],
 [0, ''],
 [0, 'Location; Jail Road Gulberg Lahore.'],
 [0, ''],
 [0, 'Shift Timing: Mon to fri ( 6 PM to 3 AM ) Night Shift'],
 [0

In [6]:
# create dictionary with values as lists
dct_lst= defaultdict(list)

for i in job_descr_txt:
    #key value pairs for default_dict_list
    dct_lst[i[0]].append(i[1])
    
dict_lst_jobsDescr=[]

for i in dct_lst.values(): # string join: lists of lists of strings
    dict_lst_jobsDescr.append(''.join(i))
    
dict_lst_jobsDescr[1]

"Education: Bachelor's/Master's Degree in management SciencesExperience: 1 year of relevant experience or fresh graduate can also be considered with good communication skills .Job ResponsibilitiesDevise and maintain office filing system\nCommunicate with students and other departments\nHandle requests and queries appropriately\nManage diary and schedule meetings and appointments\nAssist students with the application process, timetable, enrolments etc.\nManage and maintain student records, ensuring compliance with relevant data.Job Type: Full-time"

In [7]:
indeed_df['Job Description'] = dict_lst_jobsDescr
indeed_df

/tmp/ipykernel_310120/1265805867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indeed_df['Job Description'] = dict_lst_jobsDescr


,Job Title,Company,Link,Job Description
0,Microsoft Excel Expert,Progressive Resource,/rc/clk?jk=ca30fa0935fdfc32&bb=Yn3GPrx_lF8e-Dh...,Key Responsibilities:Collect data from various...
1,Admin Officer,The University of Lahore,/rc/clk?jk=14ee818fb1c66447&bb=Yn3GPrx_lF8e-Dh...,Education: Bachelor's/Master's Degree in manag...
2,Science LAB Assistant,LACAS SCHOOL NETWORK,/rc/clk?jk=075cb97c525f8753&bb=Yn3GPrx_lF8e-Dh...,Job Title: Science Lab AssistantLocation: Gulb...
3,Creative Content Writer,Adequate Solutions,/rc/clk?jk=4fe01630a3dbbaca&bb=Yn3GPrx_lF8e-Dh...,Adequate Solutions is a dynamic and innovative...
4,Associate Data Scientist,i2c inc.,/rc/clk?jk=d023edfdbd0f3669&bb=Yn3GPrx_lF8e-Dh...,Conduct exploratory data analysis (EDA) to und...
...,...,...,...,...
354,Research Analyst,TECHNEXIA,/rc/clk?jk=0f4726b73e8633ac&bb=VXjSJBZPXwnUpnN...,About the JobWe are currently seeking a talent...
355,Sr. Data Engineer (Azure Focus),RemcoSol Technologies,/rc/clk?jk=414706150bb18679&bb=VXjSJBZPXwnUpnN...,RemcoSol Technologies is seeking a talented an...
356,Nodejs Backend Developer,See Sight Tours,/rc/clk?jk=f23d34bd0d5d26e4&bb=VXjSJBZPXwnUpnN...,Node.js Developer - Firebase Functions and Goo...
357,SOC Analyst - L2,Zones,/rc/clk?jk=7faf0b0814995252&bb=VXjSJBZPXwnUpnN...,About Zones\n Zones is a global provider of en...


In [8]:
indeed_df.to_csv('indeed_jobs.csv', index=False)